In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [3]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator
from deeprank import utils

In [4]:
seed = 1234
torch.manual_seed(seed)

In [5]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
Generate numpy embed: (193368, 50)


In [6]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())
#device = torch.device("cuda")
device = torch.device("cpu")

In [7]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]
letor_config['pad_value'] = loader._PAD_

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [8]:
from deeprank import select_module
from deeprank import rank_module

In [9]:
letor_config['q_limit'] = 20
letor_config['d_limit'] = 2000
select_net = select_module.IdentityNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [10]:
'''
letor_config['q_limit'] = 20
letor_config['d_limit'] = 500
letor_config['max_match'] = 5
letor_config['win_size'] = 5
select_net = select_module.QueryCentricNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)
'''

"\nletor_config['q_limit'] = 20\nletor_config['d_limit'] = 500\nletor_config['max_match'] = 5\nletor_config['win_size'] = 5\nselect_net = select_module.QueryCentricNet(config=letor_config)\nselect_net.train()\nselect_net = select_net.to(device)\n"

In [11]:
'''
letor_config['q_limit'] = 20
letor_config['d_limit'] = 500
letor_config['max_match'] = 20
letor_config['win_size'] = 5
letor_config['finetune_embed'] = False
select_net = select_module.PointerNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)
'''

"\nletor_config['q_limit'] = 20\nletor_config['d_limit'] = 500\nletor_config['max_match'] = 20\nletor_config['win_size'] = 5\nletor_config['finetune_embed'] = False\nselect_net = select_module.PointerNet(config=letor_config)\nselect_net.train()\nselect_net = select_net.to(device)\n"

In [12]:
# letor_config['simmat_channel'] = 1
# letor_config['conv_params'] = [(8, 2, 10)]
# letor_config['fc_params'] = [50]
# letor_config['dpool_size'] = [3, 10]
# letor_config['lr'] = 0.001
# letor_config['finetune_embed'] = False
# rank_net = rank_module.MatchPyramidNet(config=letor_config)
# rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
# rank_net.train()
# optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [13]:
# letor_config['simmat_channel'] = 1
# letor_config['conv_params'] = [(8, 3, 3)]
# letor_config['fc_params'] = [200]
# letor_config['dpool_size'] = [3, 10]
# letor_config['lr'] = 0.001
# letor_config['finetune_embed'] = False
# rank_net = rank_module.MatchPyramidNet(config=letor_config)
# rank_net = rank_net.to(device)
# rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
# rank_net.train()
# optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [14]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 5, 5)]
letor_config['fc_params'] = []
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.005
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net = rank_net.to(device)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [15]:
def to_device(*variables):
    return (torch.from_numpy(variable).to(device) for variable in variables)

In [16]:
def show_text(x):
    print(' '.join([loader.word_dict[w.item()] for w in x]))

In [17]:
# X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F = \
#         pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
# X1, X1_len, X2, X2_len, Y, F = \
#         to_device(X1, X1_len, X2, X2_len, Y, F)
# show_text(X2[0])
# X1, X2_new, X1_len, X2_len_new = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
# show_text(X1[0])
# for i in range(5):
#     print(i, end=' ')
#     show_text(X2_new[0][i])

In [18]:
# X1 = X1[:1]
# X1_len = X1_len[:1]
# X2 = X2[:1]
# X2_len = X2_len[:1]
# X1_id = X1_id[:1]
# X2_id = X2_id[:1]

In [19]:
# show_text(X2[0])
# X1, X2_new, X1_len, X2_len_new = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
# show_text(X1[0])
# for i in range(5):
#     print(i, end=' ')
#     show_text(X2_new[0][i])

In [25]:
import time
start_t = time.time()
for i in range(150):
    X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F = \
        pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X1_len, X2, X2_len, Y, F = \
        to_device(X1, X1_len, X2, X2_len, Y, F)
    X1, X2, X1_len, X2_len = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
    X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
    output = rank_net(X1, X2, X1_len, X2_len, 0)
    loss = rank_net.pair_loss(output, Y)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
end_t = time.time()
print('Time Cost: %s s' % (end_t-start_t))

0.6815898418426514
0.7567062973976135
0.5823414325714111
0.6476808190345764
0.6902686953544617
0.6790058612823486
0.681659460067749
0.7014361619949341
0.7053287029266357
0.6592481732368469
0.8288141489028931
0.6558957099914551
0.7498481869697571
0.7479518055915833
0.6230575442314148
0.6434897780418396
0.6039712429046631
0.7449009418487549
0.7565931081771851
0.6968901753425598
0.7935071587562561
0.7211249470710754
0.7040175795555115
0.848415195941925
0.670654296875
0.7028505206108093
0.7429744005203247
0.7004295587539673
0.6932598352432251
0.7012687921524048
0.7168188691139221
0.5549314022064209
0.7361621260643005
0.7696697115898132
0.6200156807899475
0.7413567900657654
0.6220021843910217
0.7573643326759338
0.7620742917060852
0.7095494866371155
0.7490368485450745
0.7285387516021729
0.7047562599182129
0.6245677471160889
0.6817363500595093
0.6705864667892456
0.6920755505561829
0.7058682441711426
0.6893578171730042
0.7598142027854919
0.6605805158615112
0.7184637188911438
0.7094810605049133

In [26]:
torch.save(select_net.state_dict(), "identity.ckpt")
torch.save(rank_net.state_dict(), "matchpyramid.ckpt")

In [27]:
torch.save(select_net, "identity.model")
torch.save(rank_net, "matchpyramid.model")

In [28]:
rank_net

MatchPyramidNet(
  (embedding): Embedding(193368, 50, padding_idx=193367)
  (conv_sequential): Sequential(
    (0): Conv2d(1, 8, kernel_size=[5, 5], stride=(1, 1), padding=[2, 2])
  )
  (dpool_layer): AdaptiveMaxPool2d(output_size=[3, 10])
  (fc_sequential): Sequential()
  (out_layer): Linear(in_features=240, out_features=1, bias=True)
)

In [29]:
select_net_e = torch.load(f='identity.model')
rank_net_e = torch.load(f='matchpyramid.model')

list_gen = ListGenerator(rel_file=Letor07Path+'/relation.test.fold%d.txt'%(letor_config['fold']),
                         config=letor_config)
map_v = 0.0
map_c = 0.0

with torch.no_grad():
    for X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F in \
        list_gen.get_batch(data1=loader.query_data, data2=loader.doc_data):
        #print(X1.shape, X2.shape, Y.shape)
        X1, X1_len, X2, X2_len, Y, F = to_device(X1, X1_len, X2, X2_len, Y, F)
        X1, X2, X1_len, X2_len = select_net_e(X1, X2, X1_len, X2_len, X1_id, X2_id)
        X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
        #print(X1.shape, X2.shape, Y.shape)
        pred = rank_net_e(X1, X2, X1_len, X2_len, 0)
        map_o = utils.eval_MAP(pred.tolist(), Y.tolist())
        #print(pred.shape, Y.shape)
        map_v += map_o
        map_c += 1.0
    map_v /= map_c

print('[Test]', map_v)

[./data/letor/r5w/relation.test.fold1.txt]
	Instance size: 13652
List Instance Count: 336
[Test] 0.4388468955781538
